In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')
import statsmodels.api as sm
import tensorflow as tf
import os
from math import sqrt, ceil

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [ ]:
df_weather = pd.read_csv('dataset/weather_features.csv', parse_dates=['dt_iso'])
df_energy = pd.read_csv('dataset/energy_dataset.csv', parse_dates=['time'])

In [ ]:
df_energy.head()


In [ ]:
df_energy = df_energy.drop(['generation fossil coal-derived gas','generation fossil oil shale', 
                            'generation fossil peat', 'generation geothermal', 
                            'generation hydro pumped storage aggregated', 'generation marine', 
                            'generation wind offshore', 'forecast wind offshore eday ahead'], 
                            axis=1)


In [ ]:
df_energy.head()

In [ ]:
df_energy.describe().round(2)

In [ ]:
df_energy.dtypes

In [ ]:
df_energy['time'] = pd.to_datetime(df_energy['time'], utc=True, infer_datetime_format=True)
df_energy = df_energy.set_index('time')

In [ ]:
df_energy.info()


In [ ]:
def plot_series(df=None, column=None, series=pd.Series([]), 
                label=None, ylabel=None, title=None, start=0, end=None):

    sns.set()
    fig, ax = plt.subplots(figsize=(20, 12))
    ax.set_xlabel('Time', fontsize=16)
    if column:
        ax.plot(df[column][start:end], label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if series.any():
        ax.plot(series, label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if label:
        ax.legend(fontsize=16)
    if title:
        ax.set_title(title, fontsize=24)
    ax.grid(True)
    return ax

In [ ]:
print('There are {} missing values or NaNs in df_energy.'.format(df_energy.isnull().values.sum()))

temp_energy = df_energy.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_energy (except first occurrence) based on all columns.'
      .format(temp_energy))

In [ ]:
df_energy.isnull().sum(axis=0)

In [ ]:
ax = plot_series(df=df_energy, column='total load actual', ylabel='Total Load (MWh)',
                 title='Actual Total Load (With Nulls)', end=24*7*2)
plt.show()

In [ ]:
ax = plot_series(df=df_energy, column='generation fossil hard coal', ylabel='MWh',
                 title='Fossil Hard Coal Energy Generation (With Nulls)', end=24*7*2)
plt.show()

In [ ]:
df_energy[df_energy.isnull().any(axis=1)]

In [ ]:
print('Non-zero values in each column:\n', df_energy.astype(bool).sum(axis=0), sep='\n')

In [ ]:
df_energy.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

In [ ]:
df_energy.isnull().sum(axis=0)

In [ ]:
ax = plot_series(df=df_energy, column='total load actual', ylabel='Total Load (MWh)',
                 title='Actual Total Load (After Filling)', end=24*7*2)
plt.show()

In [ ]:
ax = plot_series(df=df_energy, column='generation fossil hard coal', ylabel='MWh',
                 title='Fossil Hard Coal Energy Generation(After Filling)', end=24*7*10)
plt.show()

In [ ]:
df_weather.head()

In [ ]:
df_weather.describe().round(2)

In [ ]:
df_weather.dtypes

In [ ]:
def df_convert_dtypes(df, convert_from, convert_to):
    cols = df.select_dtypes(include=[convert_from]).columns
    for col in cols:
        df[col] = df[col].values.astype(convert_to)
    return df

In [ ]:
df_weather = df_convert_dtypes(df_weather, np.int64, np.float64)

In [ ]:
df_weather.dtypes

In [ ]:
df_weather['time'] = pd.to_datetime(df_weather['dt_iso'], utc=True, infer_datetime_format=True)
df_weather = df_weather.drop(['dt_iso'], axis=1)
df_weather = df_weather.set_index('time')

In [ ]:
df_weather.head()

In [ ]:
df_weather.info()

In [ ]:
mean_weather_by_city = df_weather.groupby('city_name').mean()
mean_weather_by_city

In [ ]:
print('There are {} missing values or NaNs in df_weather.'.format(df_weather.isnull().values.sum()))

temp_weather = df_weather.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_weather except first occurrence based on all columns.'
      .format(temp_weather))

In [ ]:
print('There are {} rows in df_energy.'.format(df_energy.shape[0]))

cities = df_weather['city_name'].unique()
grouped_weather = df_weather.groupby('city_name')

for city in cities:
    print('There are {} rows in df_weather'
          .format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))

In [ ]:
df_weather_2 = df_weather.reset_index().drop_duplicates(subset=['time', 'city_name'], keep='last').set_index('time')

df_weather = df_weather.reset_index().drop_duplicates(subset=['time', 'city_name'], keep='first').set_index('time')

In [ ]:
print('There are {} rows in df_energy.'.format(df_energy.shape[0]))

grouped_weather = df_weather.groupby('city_name')

for city in cities:
    print('There are {} rows in df_weather'.format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))

In [ ]:
weather_description_unique = df_weather['weather_description'].unique()
weather_description_unique

In [ ]:
weather_main_unique = df_weather['weather_main'].unique()
weather_main_unique

In [ ]:
weather_id_unique = df_weather['weather_id'].unique()
weather_id_unique

In [ ]:
df_weather = df_weather.drop(['weather_description','weather_main','weather_id','weather_icon'],axis=1)

In [ ]:
df_weather.info()

In [ ]:
sns.boxplot(x=df_weather['pressure'])

In [ ]:
df_weather.loc[df_weather.pressure > 1100, 'pressure'] = np.nan
df_weather.loc[df_weather.pressure < 900, 'pressure'] = np.nan

In [ ]:
sns.boxplot(x=df_weather['pressure'])

In [ ]:
sns.boxplot(x=df_weather['temp'])

In [ ]:
sns.boxplot(x=df_weather['humidity'])

In [ ]:
sns.boxplot(x=df_weather['wind_speed'])

In [ ]:
df_weather.loc[df_weather.wind_speed > 70, 'wind_speed'] = np.nan

In [ ]:
sns.boxplot(x=df_weather['wind_speed'])

In [ ]:
sns.boxplot(x=df_weather['wind_deg'])

In [ ]:
sns.boxplot(x=df_weather['rain_1h'])

In [ ]:
print('There are {} missing values or NaNs in df_weather.'.format(df_weather.isnull().values.sum()))

In [ ]:
df_weather.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

In [ ]:
print('There are {} missing values or NaNs in df_weather.'.format(df_weather.isnull().values.sum()))

In [ ]:
temp_by_city = df_weather.pivot_table('temp', index='time', columns='city_name', aggfunc='mean')

sns.set()
fig, ax = plt.subplots(figsize=(20, 12))
temp_by_city.plot(ax=ax, subplots=True)
plt.show()

In [ ]:
## MERGE

In [ ]:
df_1, df_2, df_3, df_4, df_5 = [x for _, x in df_weather.groupby('city_name')]
dfs = [df_1, df_2, df_3, df_4, df_5]

In [ ]:
df_final = df_energy

for df in dfs:
    city = df['city_name'].unique()
    city_str = str(city).replace("'", "").replace('[', '').replace(']', '').replace(' ', '')
    df = df.add_suffix('_{}'.format(city_str))
    df_final = df_final.merge(df, on=['time'], how='outer')
    df_final = df_final.drop('city_name_{}'.format(city_str), axis=1)
    
df_final.columns

In [ ]:
df_final.head()

In [ ]:
df_final.info()

In [ ]:
print('There are {} missing values or NaNs in df_final.'.format(df_final.isnull().values.sum()))

temp_final = df_final.duplicated(keep='first').sum()

print('\nThere are {} duplicate rows in df_energy '
      .format(temp_final))

In [ ]:
df_final.shape

In [ ]:
df_final.describe()

In [ ]:
y = df_final[['total load actual','price actual']]
y.head()

In [ ]:
x = df_final[['generation biomass','generation fossil brown coal/lignite','generation fossil gas',
             'generation fossil hard coal','generation fossil oil','generation hydro pumped storage consumption','generation hydro run-of-river and poundage',
            'generation hydro water reservoir','generation nuclear','generation other','generation other renewable','generation solar',
            'generation waste','generation wind onshore','forecast solar day ahead','forecast wind onshore day ahead','temp_Barcelona',
            'temp_min_Barcelona','temp_max_Barcelona','pressure_Barcelona','humidity_Barcelona','wind_speed_Barcelona','wind_deg_Barcelona']]
x.head()

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(x,y,test_size=0.3)

In [ ]:
len(train_features)

In [ ]:
len(test_features)

In [ ]:
model = RandomForestRegressor(n_estimators = 1000,bootstrap = True)

In [ ]:
model.fit(train_features, train_labels)

In [ ]:
predictions = model.predict(test_features)
print(predictions)
print(type(predictions))
predictions.shape

In [ ]:
print(type(test_labels))
predictions.shape
test_labels = test_labels.to_numpy()
print(type(test_labels))

In [ ]:
# Calculate the absolute errors
print(type(predictions))
print(type(test_labels))
errors = (predictions - test_labels)

In [ ]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy for load:', round(accuracy, 2), '%.')

In [ ]:
import pickle
filename = 'pred_using_rf.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(test_features, test_labels)
print(result)